In [2]:
import os
import re
import pymupdf
import django_setup
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [17]:
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [18]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      "response_mime_type": "application/json",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [19]:
def clean_text(text, _normalizes=None):
    txt = text
    try:
        normalizes = _normalizes or (
            ('\n ?\d+ ?/ ?\d+ ?\n', '\n'),
            ('[ ]{2,}', ' '),
            (' \\n', '\n'),
            #('([^\.]|\S)\n(\S)', r'\1 \2'),
            ('(\w)\n(.)', r'\1 \2'),
            ('(,)\n(.)', r'\1 \2'),
            #('()\n()', r'\1 \2'),
            ('\n\n', '\n'),
            ('-\n', '-'),
            ('^\n', ''),
            ('–', '-'),
            ('•', '*'),
            ('[“”]', '"'),
            #('', ''),
            #('', ''),
            #('', ''),
            #('', ''),
            #('', ''),
        )

        for regex, new in normalizes:
            search = re.search(regex, text)
            while search:
                text = re.sub(regex, new, text)
                search = re.search(regex, text)
        return text
    except:
        return txt


In [20]:
def make_prompt(text):
    prompt = ("""    Assuma a personalidade de um especialista em Gestão Pública Municipal com mais de 10 anos de conhecimento teórico e atividade prática.
    Analise o conteúdo de <CONTEXTO></CONTEXTO> e escreva um JSON com as seguintes chaves: "temas","autores", "destinatarios", "possiveis_destinatarios", "localidades" e "analise".
    Para preencher as chaves, siga as instruções abaixo em que, respectivamente, se referem às chaves acima citadas:
1) Dentro das diversas áreas da gestão pública municipal, o texto em análise aborda a importância e faz solicitações à seus destinatários para a sociedade local. Classifique o texto criando uma lista temática concisa e coloque sua resposta à esta pergunta na chave "temas".
2) O texto em análise apresenta um ou mais autores que defendem a importância de uma ou mais ações para a sociedade. Identifique esses autores de forma objetiva, ou seja, nome e cargo. Coloque sua resposta à esta pergunta na chave "autores" no formato de lista, mesmo que seja identificado apenas um autor.
3) Se o texto em análise apresentar um ou mais destinatários das ações defendidas pelos autores. Identifique esses destinatários com seu nome, cargo e instituição a qual pertence. Explique a importância das ações defendidas pelos autores para esses destinatários no contexto de seus cargos e instituições. Utilize em sua resposta as chaves: "nome", "cargo", "instituicao", "importancia". Coloque sua resposta à esta pergunta na chave "destinatarios" no formato de lista, mesmo que seja identificado apenas um destinatário.
4) Com base nas diversas áreas da gestão pública municipal, formule uma lista de possíveis instituições destinatárias não citadas pelos autores e justifique. Use as chaves "instituicao" e "justificativa". Coloque sua resposta à esta pergunta na chave "possiveis_destinatarios" no formato de lista.
5) Caso o texto em análise apresente localidades, identifique essas localidades, sejam elas logradouros públicos, bairros ou mesmo prédios públicos e/ou privados, utilizando para isso a chave "nome", e adicionando o tipo identificado, utilizando a chave "tipo". Coloque sua resposta à esta pergunta na chave "localidades" no formato de lista.
6) Formule a defesa do texto analisado com no máximo 150 palavras sobre a importância das ações defendidas pelos autores para os destinatários identificados. É muito importante que escreva essa defesa com um texto leve de forma que uma pessoa leiga no assunto entenda. Coloque sua resposta à esta pergunta na chave "analise" no formato de markdown.

<CONTEXTO>{context}</CONTEXTO>
    """).format(context=text)

    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt


In [21]:
from sapl.materia.models import MateriaLegislativa

materias = MateriaLegislativa.objects.filter(id=21478, tipo_id=3)

for materia in materias[:1]:
    doc = pymupdf.open(materia.texto_original.path)
    text = ' '.join([page.get_text() for page in doc])
    text = clean_text(text)

prompt = make_prompt(text)

answer = model.generate_content(prompt)

In [22]:
import json

try:
    data = {}
    data['id'] = materia.id
    data['epigrafe'] = str(materia)
    data['ementa'] = materia.ementa
    data.update(json.loads(answer.text))
    data_json = json.dumps(data, indent=4, ensure_ascii=False)
    open(f'data/002_output_{materia.numero}.json', 'w').write(data_json)
except:
    print('Erro ao converter json')
    print(answer.text)
    open('data/002_error.txt', 'w').write(answer.text)

In [23]:
import yaml

yaml_string = """PROMPTs:
  - id: 2
    status: True
    temperature: 0.1
    top_p: 0.95
    top_k: 40
    text: |+
      Você é um especialista em Gestão Pública Municipal com mais de 10 anos de conhecimento teórico e atividade prática.
      Analise o conteúdo de <CONTEXTO></CONTEXTO> e escreva um JSON com as seguintes chaves: "temas", "autores", "destinatarios", "possiveis_destinatarios", "localidades" e "analise".
      Para preencher as chaves, siga as instruções abaixo em que, respectivamente, se referem às chaves acima citadas:
      1) Dentro das diversas áreas da gestão pública municipal, o texto em análise aborda a importância e faz solicitações à seus destinatários para a sociedade local. Classifique o texto criando uma lista temática concisa e coloque sua resposta à esta pergunta na chave "temas".
      2) O texto em análise apresenta um ou mais autores que defendem a importância de uma ou mais ações para a sociedade. Identifique esses autores de forma objetiva, ou seja, nome e cargo. Coloque sua resposta à esta pergunta na chave "autores" no formato de lista, mesmo que seja identificado apenas um autor.
      3) Se o texto em análise apresentar um ou mais destinatários das ações defendidas pelos autores. Identifique esses destinatários com seu nome, cargo e instituição a qual pertence. Explique a importância das ações defendidas pelos autores para esses destinatários no contexto de seus cargos e instituições. Utilize em sua resposta as chaves: "nome", "cargo", "instituicao", "importancia". Coloque sua resposta à esta pergunta na chave "destinatarios" no formato de lista, mesmo que seja identificado apenas um destinatário.
      4) Com base nas diversas áreas da gestão pública municipal, formule uma lista de possíveis instituições destinatárias não citadas pelos autores e justifique. Use as chaves "instituicao" e "justificativa". Coloque sua resposta à esta pergunta na chave "possiveis_destinatarios" no formato de lista.
      5) Caso o texto em análise apresente localidades, identifique essas localidades, sejam elas logradouros públicos, bairros ou mesmo prédios públicos e/ou privados, utilizando para isso a chave "nome", e adicionando o tipo identificado, utilizando a chave "tipo". Coloque sua resposta à esta pergunta na chave "localidades" no formato de lista.
      6) Formule a defesa do texto analisado com no máximo 150 palavras sobre a importância das ações defendidas pelos autores para os destinatários identificados. É muito importante que escreva essa defesa com um texto leve de forma que uma pessoa leiga no assunto entenda. Coloque sua resposta à esta pergunta na chave "analise" no formato de markdown.

      <CONTEXTO>{context}</CONTEXTO>"""

yaml_data = yaml.load(yaml_string, Loader=yaml.FullLoader)

In [24]:
yaml_data

{'PROMPTs': [{'id': 2,
   'status': True,
   'temperature': 0.1,
   'top_p': 0.95,
   'top_k': 40,
   'text': 'Você é um especialista em Gestão Pública Municipal com mais de 10 anos de conhecimento teórico e atividade prática.\nAnalise o conteúdo de <CONTEXTO></CONTEXTO> e escreva um JSON com as seguintes chaves: "temas", "autores", "destinatarios", "possiveis_destinatarios", "localidades" e "analise".\nPara preencher as chaves, siga as instruções abaixo em que, respectivamente, se referem às chaves acima citadas:\n1) Dentro das diversas áreas da gestão pública municipal, o texto em análise aborda a importância e faz solicitações à seus destinatários para a sociedade local. Classifique o texto criando uma lista temática concisa e coloque sua resposta à esta pergunta na chave "temas".\n2) O texto em análise apresenta um ou mais autores que defendem a importância de uma ou mais ações para a sociedade. Identifique esses autores de forma objetiva, ou seja, nome e cargo. Coloque sua respost

##### Extração de Temas gerado pelo código acima implementado já no PortalCMJ

In [49]:
from sapl.base.models import Metadata
temas_global = set()
mds = Metadata.objects.all()

for md in mds:
    temas = md.metadata.get('genia', {}).get('temas', [])
    for tema in temas:
        if tema not in temas_global:
            temas_global.add(' '.join(map(lambda x: x.capitalize() if len(x) > 3 else x, tema.split())))

temas_global = {
    k: k for k in sorted(temas_global)
    }

In [50]:
temas_global = list(temas_global.items())

In [51]:
temas_global[0]

('Abastecimento de Água', 'Abastecimento de Água')

In [56]:
md = Metadata.objects.filter(metadata__genia__temas__icontains=temas_global[0][0]).first()

In [57]:
md.metadata

{'genia': {'temas': ['Infraestrutura urbana',
   'Abastecimento de água',
   'Saneamento básico',
   'Qualidade de vida',
   'Saúde pública'],
  'analise': 'O vereador Guilherme Alves, preocupado com a falta de água potável no setor Nova Esperança, em Jataí, solicita ao Vice-governador de Goiás e ao Presidente da SANEAGO a construção de uma rede de abastecimento. A falta dessa infraestrutura obriga os moradores a buscar alternativas como cisternas, que podem ser contaminadas. A construção da rede é essencial para garantir a saúde e qualidade de vida da população, pois o acesso à água potável é um direito básico. A ação do vereador busca unir forças para levar essa melhoria aos moradores, mostrando a importância da colaboração entre os poderes públicos para atender às necessidades da comunidade.',
  'autores': [{'nome': 'Guilherme Alves', 'cargo': 'Vereador'}],
  'model_name': 'gemini-2.0-flash-exp',
  'localidades': [{'nome': 'Setor Nova Esperança', 'tipo': 'Bairro'},
   {'nome': 'Jata